In [2]:
pip install numpy pandas scipy statsmodels matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

In [18]:
#Необходимо прочитать файл, чтобы узнать какие есть первоначальные данные
df = pd.read_csv('C:/Data Science/2_Task/abtest/ab_test_ensembles.csv', delimiter=',')
#Скопируем первоначальный датафрейм, чтобы не портить оригинал
df_copy=df.copy()
df_copy.info()
df_copy.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    294478 non-null  object
 1   timestamp  294478 non-null  object
 2   group      294478 non-null  object
 3   variants   294478 non-null  object
 4   converted  294478 non-null  int64 
 5   location   294478 non-null  object
 6   age_group  294478 non-null  object
 7   user_type  294478 non-null  object
dtypes: int64(1), object(7)
memory usage: 18.0+ MB


,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user


In [16]:
df_copy.isnull().sum()

user_id      0
timestamp    0
group        0
variants     0
converted    0
location     0
age_group    0
user_type    0
dtype: int64

In [20]:
# Убедимся, что контрольная группа и испытуемая видят старую страницу и новую -> https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html
pd.crosstab(df_copy['group'], df_copy['variants'])

variants,standard,svm_ensemble_v_1_22
group,,
control,145274,1928
treatment,1965,145311


In [21]:
#Проверка на дубликаты. Т.е. нам необходимо, чтобы пользователь видел или старую страницу, или новую, но не 2 сразу
page = df_copy['user_id'].value_counts(ascending=False)
duplicate_users = page[page > 1].count()
print(f"У нас имеются {duplicate_users} пользователей, которые не уникальны в нашей выборке.")

У нас имеются 0 пользователей, которые не уникальны в нашей выборке.


In [ ]:
#На основании этого получается, что у нас все пользователи уникальны

In [24]:
# Считаем размер эффекта -> https://www.statsmodels.org/dev/generated/statsmodels.stats.proportion.proportion_effectsize.html
effect_size = sms.proportion_effectsize(0.13, 0.15)
# Cчитаем необходимый размер выборки -> https://www.statsmodels.org/dev/generated/statsmodels.stats.power.NormalIndPower.html?highlight=normalindpower
# https://www.statsmodels.org/dev/generated/statsmodels.stats.power.NormalIndPower.solve_power.html#statsmodels.stats.power.NormalIndPower.solve_power
required_n = sms.NormalIndPower().solve_power(effect_size, power=0.8, alpha=0.05, ratio=1) # Calculating sample size needed
# Округляем до ближайшего целого
required_n = ceil(required_n)
print("Для каждой группы нам необходимо не меньше ",required_n, " наблюдений")

Для каждой группы нам необходимо не меньше  4720  наблюдений


In [25]:
#Формируем случайную выборку
#Не до конца поняла что подразумевает random_state !Документацию читала, но не совсем понятно
control_sample = df_copy[df_copy['group'] == 'control'].sample(n = required_n, random_state=42)
treatment_sample = df_copy[df_copy['group'] == 'treatment'].sample(n = required_n, random_state=42)

# Сформируем общую случайную выборку
ab_df = pd.concat([control_sample, treatment_sample], axis=0)
ab_df.reset_index(drop=True, inplace=True)
ab_df.head()

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,bcf8452b-3056-4d45-994f-5f286fd9869a,2022-05-18 04:44:18.590293,control,standard,0,United Kingdom,42-49,new_user
1,3c827a98-0868-4ce5-ac31-8195e243af37,2022-05-14 05:22:43.771772,control,standard,0,United Kingdom,50-57,new_user
2,71c72bb9-13d9-4755-b398-b2c5d8b1c4d6,2022-05-23 08:52:53.050325,control,standard,0,United Kingdom,34-41,new_user
3,654ee878-b0c5-4223-b9fb-995ca3d2eaf1,2022-05-07 23:27:09.109877,control,standard,0,United Kingdom,26-33,registered_user
4,1e044366-7e86-4d92-bba7-3afaa6f887f8,2022-05-20 14:26:10.133979,control,standard,0,United Kingdom,42-49,new_user


In [26]:
conversion_rates = ab_df.groupby('group')['converted']

# Стандартное отклонение для сформированной выборки
std_proportion = lambda x: np.std(x, ddof=0)
# Стандартная ошибка для сформированной выборки, формулу стандартной ошибки вы знаете из лекций. Забыли - погуглите.
se_proportion = lambda x: stats.sem(x, ddof=0)

conversion_rates = conversion_rates.agg([np.mean, std_proportion, se_proportion]).rename(columns={'mean':'CR mean', '<lambda_0>':'CR std', '<lambda_1>':'CR se'})
conversion_rates

,CR mean,CR std,CR se
group,,,
control,0.132415,0.338942,0.004933
treatment,0.119915,0.324862,0.004729


In [ ]:
#Как мы видим по метрикам, два эксперимента +/- по выборочным статистикам показывают себя одинаково, за исключением выборочного среднего коэффициента конверсии ~11.9% против ~13.2%

Действительно ли разница между двумя группами испытуемых является статистически значимой? И какой же вариант нам стоит оставить?
Сформулируем гипотезу исследования:
Ho(нулевая гипотеза)  наш новый сервис не оказал никакого воздействия на уровень конверсии, а среднее значение уровня конверсии ничем не отличается от метрики при использовании старой версии сервиса.

H1(альтернативная гипотеза)  наш новый сервис оказал на пользовательскую активность (какую-то метрику) нашей аудитории, а среднее значение метрики ничем не отличается от метрики при использовании старой версии сервиса.

Теперь нам надо проверить нулевую гипотезу, для этого определить ровень альфы 0.05 (еще в курсе по статистике его называют p-уровень значимости)

In [27]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control_results = ab_df[ab_df['group'] == 'control']['converted']
treatment_results = ab_df[ab_df['group'] == 'treatment']['converted']

n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'Z-statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'CI 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'CI 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

Z-statistic: 1.83
p-value: 0.067
CI 95% for control group: [0.123, 0.142]
CI 95% for treatment group: [0.111, 0.129]


Расшифровка полученных результатов

Так как наше p-значение = 0.06 что является близким по сравнению с выбранной альфой 0.05, то согласно статистическому выводу, мы не можем отклонить нулевую гипотезу H0, что означает наш сервис никак статистически значимо не влияет на уровень конверсии и повышение уровня конверсии - это случайность.

Доверительный интервал для испытуемой группы [0.111, 0.129] или 11,1% и 12,9% соответственно:

Он включает наш baseline (я бы сказала близок)
Он не достигает нашей цели 15%
Мы можем интерпритировать это как - наиболее вероятно, что реальный уровень конверсии сайта с новым дизайном будет близок к baseline чем к 15%, который мы определили по нашей цели. Таким образом у нас два выбора:

Так как мы будем близко к baseline то в случае если сайт воспринимается "красиво" можно его оставить.
Можно создать новую версию (потратить время, деньги, нервы) и опять протестировать, надо решать стоят ли затраты на новую версию 2% улучшения (покроются ли затраты на работу) - задача уже финансов, а может и вас :) Считается коэффициент окупаемости и ROI. Если да - можем попробовать еще раз.


Мои вопросы и размышления по AB-тесту:
1. Я просмотрела все разделы и курсы по AB-тестированию по вашим рекомендациям и считаю, что в данном случае нет смысла выводить какие либо графики, т.к. у нас либо 1 либо 0 характеритика, поэтому выбросы тут не будут. А вот если бы было среднее значение или значения например плотности или иного физического показателя, то я бы вывела график QQ plot. Если я неправильно трактую, прошу тогда уточнить и пояснить.
